<a href="https://colab.research.google.com/github/Dev-hottae/PINC/blob/master/Data_Analysis/Train_KoGPT2_Summarizer(Chatbot).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls drive/'My Drive'/'Colab Notebooks'/

 __MACOSX	  'report.ipynb의 사본의 사본'	     requirements.txt
 NarrativeKoGPT2  'report.ipynb의 사본의 사본 (1)'   Untitled0.ipynb


In [2]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/requirements.txt

     |████████████████████████████████| 348kB 2.6MB/s 
     |████████████████████████████████| 55.0MB 55kB/s 
     |████████████████████████████████| 1.1MB 48.6MB/s 
     |████████████████████████████████| 1.3MB 46.7MB/s 
     |████████████████████████████████| 2.9MB 21.9MB/s 
     |████████████████████████████████| 890kB 47.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588539 sha256=af43c4ff58f5406511031348cc1d782bd59ba07271271f2fbe75d1dbe424423d
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
  Created wheel for kss: filename=kss-1.3.1-cp36-cp36m-linux_x86_64.whl size=251564 sha256=f63f57ecb39400946345d9d0cc0f4c9aa3a7bcd51b17053385f011b46aecacc2
  Stored in directory: /root/.cache/pip/wheels/8b/98/d1/53f75f89925cd95779824778725ee3fa36e7aa55ed26ad54a8
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bc0f5ea4db1a8c12713d9049e7cb3ebef

In [3]:
!pip install git+https://github.com/SKT-AI/KoGPT2.git

  Cloning https://github.com/SKT-AI/KoGPT2.git to /tmp/pip-req-build-8c07hr6y
  Running command git clone -q https://github.com/SKT-AI/KoGPT2.git /tmp/pip-req-build-8c07hr6y
  Created wheel for kogpt2: filename=kogpt2-0.1.1-cp36-none-any.whl size=14054 sha256=2540bcb62fe5a78e04d31e73ebec9f02e1168e75e394a1e7b0883ff9799ef16e
  Stored in directory: /tmp/pip-ephem-wheel-cache-jctobdga/wheels/3b/a2/30/432bb7490a2ea23a90049e6c5725f6acd7e925f1abfb3d7ddf
Successfully built kogpt2


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from gluonnlp.data import SentencepieceTokenizer 
import gluonnlp as nlp
import pandas as pd
import mxnet as mx
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from mxnet import gluon, nd
from mxnet.gluon import nn

In [6]:
from kogpt2.utils import download, tokenizer, get_tokenizer
from kogpt2.pytorch_kogpt2 import GPT2Config, GPT2LMHeadModel

In [7]:
#GPU 사용
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [8]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
save_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/checkpoint/'

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000,
    "output_past": None
}

In [9]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=None,
                                              config=GPT2Config.from_dict(kogpt2_config),
                                              state_dict=torch.load(model_path))

device = torch.device(ctx)
kogpt2model.to(device)
# Fine Tunning을 위해 train 선언
kogpt2model.train()
# 단어 뭉치 가져오기
vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')

In [11]:
tok_path = get_tokenizer()
vocab = vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path, 0, 0)

using cached model


In [12]:
data_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/DataSet/Sentence_Word.csv'
news_data = pd.read_csv(data_path, encoding='utf-8')
news_data.head()

,Sentence,Word
0,역시 큰 사고 때면 어김없이 동포애를 재확인할 수 있군요.,사고 때 동포 애 확인 수
1,이번 지진ㆍ해일 참사에 희생된 한국인들과 수색작업에 참가한 사람들을 도우려는 온정의...,이번 지진 참사 희생 한국인 수색 작업 참가 사람 온정 손길 것 현장 지휘 본부 관...
2,푸껫 시내 로열 푸껫 시티 호텔 2층에 마련된 현장지휘본부에는 새해 첫날에도 불구하...,푸껫 시내 로열 푸껫 시티 호텔 층 마련 현장 지휘 본부 새해 첫날 현지 교민 자원...
3,또 이번 사태로 비상근무체제에 돌입한 방콕의 한국대사관과 한인회 등에도 위문품 전달...,이번 사태 비상근무 체제 돌입 방콕 한국 대사관 인회 등 위문품 전달 방법 문의 전화
4,푸껫에 거주하는 800여 명의 교민들은 이번 사태로 주업인 관광업이 한동안 상당한 ...,푸껫 거주 명 교민 이번 사태 주업 관광업 한동안 상당 타격 것 우려 채 한국인 희...


In [13]:
dataset_train = []
for i in tqdm(range(len(news_data))):
    dataset_train.append([news_data['Word'][i], news_data['Sentence'][i]])

dataset_train[:5]

100%|██████████| 722/722 [00:00<00:00, 65263.41it/s]


[['사고 때 동포 애 확인 수', '역시 큰 사고 때면 어김없이 동포애를 재확인할 수 있군요.'],
 ['이번 지진 참사 희생 한국인 수색 작업 참가 사람 온정 손길 것 현장 지휘 본부 관계자 말',
  '이번 지진ㆍ해일 참사에 희생된 한국인들과 수색작업에 참가한 사람들을 도우려는 온정의 손길이 끊이지 않은 것을 지켜본 현장지휘본부 관계자의 말이다.'],
 ['푸껫 시내 로열 푸껫 시티 호텔 층 마련 현장 지휘 본부 새해 첫날 현지 교민 자원 봉사자 진출 기업체 관계자 등 방문',
  '푸껫 시내 로열 푸껫 시티 호텔 2층에 마련된 현장지휘본부에는 새해 첫날에도 불구하고 현지 교민 자원봉사자, 진출 기업체 관계자 등의 방문이 잇따랐다.'],
 ['이번 사태 비상근무 체제 돌입 방콕 한국 대사관 인회 등 위문품 전달 방법 문의 전화',
  '또 이번 사태로 비상근무체제에 돌입한 방콕의 한국대사관과 한인회 등에도 위문품 전달 방법을 문의하는 전화가 빗발쳤다.'],
 ['푸껫 거주 명 교민 이번 사태 주업 관광업 한동안 상당 타격 것 우려 채 한국인 희생자 수색 작업 원활 진행 통역 안내 등 자원 봉사',
  '푸껫에 거주하는 800여 명의 교민들은 이번 사태로 주업인 관광업이 한동안 상당한 타격이 불가피할 것이라는 우려를 잠시 뒤로한 채 한국인 희생자 수색작업을 원활하게 진행하기 위해 통역, 안내 등의 자원봉사를 아끼지 않고 있다.']]

In [14]:
class GPT2Dataset(Dataset):
    def __init__(self, data_file, vocab, tokenizer, max_len=64):
        self.data = data_file
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.padder = nlp.data.PadSequence(self.max_len, pad_val=vocab[vocab.padding_token])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        input = self.data[index][0]
        answer = self.data[index][1]
        i_toked = ['<usr>', ] + self.tokenizer(input) + ['</s>', ]
        i_len = len(i_toked)
        a_toked = ['<sys>', ] + self.tokenizer(answer) + ['</s>', ]
        a_len = len(a_toked)
        if i_len + a_len > self.max_len:
            a_len = self.max_len - i_len
            if a_len <= 0:
                i_toked = i_toked[-(int(self.max_len/2)):]
                i_toked = len(i_toked)
                a_len = self.max_len - i_len
                assert a_len > 0
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)
            assert a_len == len(a_toked), f'{a_len} ==? {len(a_toked)}'
        mask = [0] * i_len + [1] * a_len + [0] * (self.max_len - i_len - a_len)
        
        return (self.padder(vocab[i_toked + a_toked]), nd.array(mask))

In [15]:
batch_size = 2
news_dataset = GPT2Dataset(dataset_train, vocab, sentencepieceTokenizer)
news_data_loader = mx.gluon.data.DataLoader(news_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=3)

In [16]:
for i in news_data_loader:
    print(i)
    break

[
[[    2  3714 10753  1953  1889  3133  1440   509   666  5592  2861   710
   7073 13607   133  2127   470  1906  1440   197     1     4 13509 18351
  47653 47476  1813  1953  5475  2047  1889  7096 22139 18148   401 47624
   9837 47623 47769 47624  2100   710 47623 47671  7073 41058 13494   299
   9504   470 47460  5447  1558  7953  1012   123 47452   445 12976   123
  47440     1     3     3]
 [    2   787  2482  4241  8792  1110  1665  2611   447  4241   509   389
    903  2143   683  1267     1     4   787 24209   211  4241   108 15138
   1110 47443  1665 37580   494  2966  9349  5904  4241  4230  9607   389
    903   808 47453  2143 47463   683  6102 27127   202 47440     1     3
      3     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3]]
<NDArray 2x64 @cpu_pinned(0)>, 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 

In [17]:
class KoGPT2Chat(nn.HybridBlock):
    def __init__(self, kogpt2, prefix=None, params=None):
        super(KoGPT2Chat, self).__init__(prefix=prefix, params=params)
        self.kogpt2 = kogpt2

    def hybrid_forward(self, F, inputs):
        # (batch, seq_len, hiddens)
        output, _ = self.kogpt2(inputs)
        return output

In [29]:
kogptqa = KoGPT2Chat(kogpt2model)
kogptqa.hybridize()

In [24]:
learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(kogpt2model.parameters(), lr=learning_rate)

In [27]:
epoch = 10
for epoch in range(epoch):
    count = 0
    avg_loss = (0.0, 0.0)
    for batch_id, (token_ids, mask) in tqdm(news_data_loader):
        if step_num < num_warmupkogptqa = KoGPT2Chat(model)
    kogptqa.hybridize()_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            non_warmup_steps = step_num - num_warmup_steps
            offset = non_warmup_steps / (num_train_steps -
                                            num_warmup_steps)
            new_lr = lr - offset * lr
    #     trainer.set_learning_rate(new_lr)
    #     with mx.autograd.record():
    #         # load data to GPU or GPU
    #         token_ids = token_ids.as_in_context(ctx)
    #         mask = mask.as_in_context(ctx)
    #         label = label.as_in_context(ctx)
    #         # forward computation
    #         out = kogptqa(token_ids)
    #         masked_out = nd.where(
    #             mask.expand_dims(axis=2).repeat(repeats=out.shape[2],
    #                                             axis=2), out,
    #             neg * nd.ones_like(out))
    #         # loss for responses exincluding MASK and PAD
    #         ls = loss_function(masked_out, label).sum() / mask.sum()        
        
    #     optimizer.zero_grad()
    #     # Data에 Torch 스택
    #     data = torch.stack(data)
    #     data = data.transpose(1,0)
    #     # 데이터와 모델에 GPU 설정
    #     data = data.to(device)
    #     kogpt2model = kogpt2model.to(device)
    #     # 결과값
    #     outputs = kogpt2model(data, labels=data)
    #     loss, logits = outputs[:2]
    #     loss = loss.to(device)
    #     loss.backward()
    #     avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
    #     optimizer.step()
    #     count+=1

    #     if count % 10 == 0:
    #         print('epoch no.{0} train no.{1}  loss = {2:.5f} avg_loss = {3:.5f}' . format(epoch+1, count, loss, avg_loss[0] / avg_loss[1]))

    # # 10에폭 단위 체크포인트 저장
    # if epoch % 10 == 0:
    #     torch.save({
    #         'model_state_dict': kogpt2model.state_dict(),
    #         'optimizer_state_dict' : optimizer.state_dict(),
    #         'loss' : loss
    #         }, save_path+'summarize_KoGPT2_checkpoint{}.tar'.format(epoch//10))

  0%|          | 0/361 [00:00<?, ?it/s]


NameError: ignored

In [23]:
for epoch_id in range(10):
    step_loss = 0
    for batch_id, (token_ids, mask) in enumerate(news_data_loader):
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            non_warmup_steps = step_num - num_warmup_steps
            offset = non_warmup_steps / (num_train_steps -
                                            num_warmup_steps)
            new_lr = lr - offset * lr
        trainer.set_learning_rate(new_lr)
        with mx.autograd.record():
            # load data to GPU or GPU
            token_ids = token_ids.as_in_context(ctx)
            mask = mask.as_in_context(ctx)
            label = label.as_in_context(ctx)
            # forward computation
            out = kogptqa(token_ids)
            masked_out = nd.where(
                mask.expand_dims(axis=2).repeat(repeats=out.shape[2],
                                                axis=2), out,
                neg * nd.ones_like(out))
            # loss for responses exincluding MASK and PAD
            ls = loss_function(masked_out, label).sum() / mask.sum()
        # backward computation
        ls.backward()
        if not accumulate or (batch_id + 1) % accumulate == 0:
            trainer.allreduce_grads()
            nlp.utils.clip_grad_global_norm(params, 1)
            trainer.update(accumulate if accumulate else 1)
            step_num += 1
            if accumulate and accumulate > 1:
                # set grad to zero for gradient accumulation
                all_model_params.zero_grad()
        step_loss += ls.asscalar()
        if step_num % log_interval == 0 and step_num > 0:
            print(
                '[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.10f}, train ppl={:.3f}'
                .format(epoch_id + 1, batch_id + 1, len(train_dataloader),
                        step_loss / log_interval, trainer.learning_rate,
                        math.exp(step_loss / log_interval)))
            step_loss = 0
    torch.save({
            'model_state_dict': kogpt2model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'loss' : loss
            }, save_path+'summarize_KoGPT2_checkpoint{}.tar'.format(epoch//10))

NameError: ignored